# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298317709087                   -0.85    4.9
  2   -8.300167262678       -2.73       -1.25    1.0
  3   -8.300446998201       -3.55       -1.89    3.6
  4   -8.300462038895       -4.82       -2.77    1.6
  5   -8.300464160673       -5.67       -3.03   11.5
  6   -8.300464396437       -6.63       -3.21    1.0
  7   -8.300464519291       -6.91       -3.35    2.2
  8   -8.300464581237       -7.21       -3.48    1.0
  9   -8.300464630032       -7.31       -3.69    1.0
 10   -8.300464638701       -8.06       -3.86    1.0
 11   -8.300464643618       -8.31       -4.20    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67458309243                   -0.70    6.8
  2   -16.67861974083       -2.39       -1.14    1.0
  3   -16.67922800601       -3.22       -1.87    3.5
  4   -16.67928052096       -4.28       -2.77    4.4
  5   -16.67928595128       -5.27       -3.22    5.0
  6   -16.67928620814       -6.59       -3.52    3.4
  7   -16.67928622034       -7.91       -3.96    1.0
  8   -16.67928622394       -8.44       -4.60    1.2


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32765689747                   -0.56    6.6
  2   -33.33464569829       -2.16       -1.00    1.4
  3   -33.33602596987       -2.86       -1.73    7.8
  4   -33.33616556432       -3.86       -2.64    5.8
  5   -33.33687324506       -3.15       -2.41    6.8
  6   -33.33690168848       -4.55       -2.52    2.4
  7   -33.33694331515       -4.38       -3.69    2.9
  8   -33.33694381642       -6.30       -3.92    5.8
  9   -33.33694391864       -6.99       -4.42    3.0


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298087659975                   -0.85    5.4
  2   -8.300258187008       -2.66       -1.59    1.0
  3   -8.300430063739       -3.76       -2.42    5.4
  4   -8.300324480610   +   -3.98       -2.18    7.0
  5   -8.300464473504       -3.85       -3.66    3.0
  6   -8.300464566261       -7.03       -3.84    2.2
  7   -8.300464631581       -7.18       -4.24    2.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32578091010                   -0.56    7.5
  2   -33.31874250559   +   -2.15       -1.27    1.4
  3   -16.71347227376   +    1.22       -0.45    7.8
  4   -33.32465682876        1.22       -1.91    6.6
  5   -33.31117978158   +   -1.87       -1.69    3.2
  6   -32.86739606834   +   -0.35       -1.22    4.5
  7   -33.30540993844       -0.36       -1.71    4.9
  8   -33.33619032110       -1.51       -2.51    3.1
  9   -33.33645420722       -3.58       -2.64    3.6
 10   -33.33678660048       -3.48       -2.65    2.8
 11   -33.33683309009       -4.33       -3.01    2.1
 12   -33.33694242715       -3.96       -3.57    3.1
 13   -33.33694233766   +   -7.05       -3.67    4.4
 14   -33.33694361158       -5.89       -4.10    2.1


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.